In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[24]:



import os, pickle, time, re, sys, operator

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import auc, classification_report, roc_auc_score, f1_score, matthews_corrcoef, balanced_accuracy_score, r2_score , confusion_matrix, precision_score, recall_score

import pandas as pd
import numpy as np
import seaborn as sns

from my_util import *
from lime.lime.lime_tabular import LimeTabularExplainer

# from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import matplotlib.pyplot as plt


from datetime import datetime
from collections import Counter


sys.path.append(os.path.abspath('../'))
from pyexplainer.pyexplainer_pyexplainer import PyExplainer

from IPython.display import display



# In[25]:


data_path = './dataset/'
result_dir = './new_eval_result/'
dump_dataframe_dir = './dump_df/'
pyExp_dir = './pyExplainer_obj/'
other_object_dir = './other_object/'
# proj_name = 'qt' # ['openstack','qt']


# In[26]:


# def test_lime(proj_name):
#     global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name)
#     all_eval_result = pd.DataFrame()
    
#     for i in range(0,len(feature_df)):
#         X_explain = feature_df.iloc[[i]]

#         row_index = str(X_explain.index[0])

#         py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'.pkl','rb'))
#         lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))

#         # this data can be used for both local and global model
#         py_exp_synthetic_data = py_exp['synthetic_data'].values
#         # this data can be used with global model only
#         lime_exp_synthetic_data = lime_exp['synthetic_instance_for_global_model']
#         # this data can be used with local model only
#         lime_exp_synthetic_data_local = lime_exp['synthetic_instance_for_lobal_model']
        
#         display(X_explain)
#         display(lime_exp_synthetic_data[:5,:])
#         display(lime_exp_synthetic_data_local[:5,:])
        
#         break
        
# test_lime('openstack')


# In[27]:


flip_sign_dict = {
    '<': '>=',
    '>': '<=',
    '=': '!=',
    '>=': '<',
    '<=': '>',
    '!=': '=='
}

'''
    input: rule (str)
'''
def flip_rule(rule):
    rule = re.sub(r'\b=\b',' = ',rule) # for LIME
#     rule = rule.replace('&','and') # for RuleFit
    found_rule = re.findall('.* <=? [a-zA-Z]+ <=? .*', rule) # for LIME
    ret = ''
    
    # for LIME that has condition like this: 0.53 < nref <= 0.83
    if len(found_rule) > 0:
        found_rule = found_rule[0]
    
        var_in_rule = re.findall('[a-zA-Z]+',found_rule)

        var_in_rule = var_in_rule[0]
        
        splitted_rule = found_rule.split(var_in_rule)
        splitted_rule[0] = splitted_rule[0] + var_in_rule # for left side
        splitted_rule[1] = var_in_rule + splitted_rule[1] # for right side
        combined_rule = splitted_rule[0] + ' or ' + splitted_rule[1]
        ret = flip_rule(combined_rule)
        
    else:
        for tok in rule.split():
            if tok in flip_sign_dict:
                ret = ret + flip_sign_dict[tok] + ' '
            else:
                ret = ret + tok + ' '
    return ret

def get_top_k_global_features(global_model, indep, top_k_global_feature_num = 5):
    global_feature_df = pd.DataFrame()
    global_feature_df['feature'] = indep
    global_feature_df['importance'] = global_model.feature_importances_

    global_feature_df = global_feature_df.sort_values(by='importance',ascending=False)

    top_k_global_features = list(global_feature_df['feature'])[:top_k_global_feature_num]

    return top_k_global_features
    
def sort_global_feature(global_model, indep):
    global_feature_df = pd.DataFrame()
    global_feature_df['feature'] = indep
    global_feature_df['importance'] = global_model.feature_importances_

    global_feature_df = global_feature_df.sort_values(by='importance',ascending=False)

    sorted_global_features = list(global_feature_df['feature'])

    return sorted_global_features

def get_rule_str_of_rulefit(local_rulefit_model):
    rule_df = local_rulefit_model.get_rules()
#     print(rule_df)
    top_k = 5
    top_k_positive_rules = rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("importance", ascending=False).head(top_k)
#     top_k_positive_rules = rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("coef", ascending=False).head(top_k)

    the_best_defective_rule_str = list(top_k_positive_rules['rule'])[0]
    
    return the_best_defective_rule_str

def get_rule_str_of_rulefit_new_version(local_rulefit_model):
    rule_df = local_rulefit_model.get_rules()
    rule_df =  rule_df[(rule_df.coef > 0) & (rule_df.type=='rule')].sort_values("importance", ascending=False)
    
    rule_list = list(rule_df['rule'])
    dup_feature_in_rule = [] # true or false...
    
    for r in rule_list:
        var_in_rule = re.findall('[a-zA-Z]+', r)
        var_count = Counter(var_in_rule)
        max_count = max(list(var_count.values()))
        
        if max_count > 1:
            dup_feature_in_rule.append(True)
        else:
            dup_feature_in_rule.append(False)
           
    if False not in set(dup_feature_in_rule):
#         print('wtf')
        rule_df = rule_df.head(5)
        the_best_defective_rule_str = list(rule_df['rule'])[0]
        
    else:
        rule_df['contain_dup_var'] = dup_feature_in_rule    
        the_best_defective_rule_str = rule_df[rule_df['contain_dup_var']==False].iloc[0]['rule']
    
    return the_best_defective_rule_str

def aggregate_list(l):
    return np.mean(l), np.median(l)


# In[28]:


# def test_new_rule_from_rulefit(proj_name):
#     global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name)
#     all_eval_result = pd.DataFrame()
    
#     c = 0
    
# #     py_exp_all_rules = []
# #     lime_all_rules = []
#     py_exp_all_vars = []
#     lime_all_vars = []
    
#     print('global feature feature importance ranking:')
#     print(sort_global_feature(global_model, indep))
#     for i in range(0,len(feature_df)):
#         X_explain = feature_df.iloc[[i]]

#         row_index = str(X_explain.index[0])

#         py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'_20_rules.pkl','rb'))
#         py_exp_local_model = py_exp['local_model']
        
#         lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))
        
# #         py_exp_rule = get_rule_str_of_rulefit(py_exp_local_model)
#         py_exp_rule_new = get_rule_str_of_rulefit_new_version(py_exp_local_model)
#         lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

#         py_exp_pred = eval_rule(py_exp_rule_new, X_explain)[0]
#         lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

#         if py_exp_pred == 1:
#             py_exp_var_in_rule = list(set(re.findall('[a-zA-Z]+', py_exp_rule_new)))
#             py_exp_all_vars.extend(py_exp_var_in_rule)
#         if lime_pred == 1:
#             lime_var_in_rule = list(set(re.findall('[a-zA-Z]+', lime_the_best_defective_rule_str)))
#             lime_all_vars.extend(lime_var_in_rule)
            
# #         py_exp_all_rules.append(py_exp_rule_new)
# #         lime_all_rules.append(lime_the_best_defective_rule_str)
        
# #         eval_result = eval_rule(lime_the_best_defective_rule_str, X_explain)

# #         if eval_result[0]:
# #             c =c+1

    
# #     print(len(set(py_exp_all_rules)))
# #     print(len(set(lime_all_rules)))
    
#     print('pyExplainer var count')
#     print(Counter(py_exp_all_vars))
#     print('-'*100)
#     print('LIME var count')
#     print(Counter(lime_all_vars))
    
# print('openstack')
# test_new_rule_from_rulefit('openstack')
# print('*'*100)
# print('qt')
# test_new_rule_from_rulefit('qt')


# In[29]:


def is_in_top_k_global_features(top_k_global_features, the_best_defective_rule_str):
    # remove numeric value
    new_the_best_defective_rule_str = re.sub('\d+','', the_best_defective_rule_str)

    # remove special characters
    new_the_best_defective_rule_str = re.sub('\W+',' ',new_the_best_defective_rule_str)
    splitted_rule = new_the_best_defective_rule_str.split()

    local_feature_count = 0
    
    found_features = set(splitted_rule).intersection(top_k_global_features)
    return list(found_features)

# def eval_rule(rule, X_explain):
#     var_in_rule = re.findall('[a-zA-Z]+',rule)
#     rule = rule.replace('&','and') # just for rulefit
#     rule = re.sub(r'\b=\b','==',rule)
# #             rule = rule.replace('=','==')

#     var_dict = {}

#     for var in var_in_rule:
#         var_dict[var] = float(X_explain[var])

#     eval_result = eval(rule,var_dict)
#     return eval_result

        
def prepare_data_for_testing(proj_name, global_model_name = 'RF'):
    global_model_name = global_model_name.upper()
    global_model = pickle.load(open(proj_name+'_'+global_model_name+'_global_model.pkl','rb'))

    correctly_predict_df = pd.read_csv(dump_dataframe_dir+proj_name+'_'+global_model_name+'_correctly_predict_as_defective.csv')
    correctly_predict_df = correctly_predict_df.set_index('commit_id')

    dep = 'defect'
    indep = correctly_predict_df.columns[:-3] # exclude the last 3 columns

    feature_df = correctly_predict_df.loc[:, indep]
    
    return global_model, correctly_predict_df, indep, dep, feature_df
    


# # Global model evaluation

# In[30]:


def get_prediction_result_df(proj_name, global_model_name):
    global_model_name = global_model_name.upper()
    if global_model_name not in ['RF','LR']:
        print('wrong global model name. the global model name must be RF or LR')
        return
    
    prediction_df_dir = dump_dataframe_dir+proj_name+'_'+global_model_name+'_prediction_result.csv'
    correctly_predict_df_dir = dump_dataframe_dir+proj_name+'_'+global_model_name+'_correctly_predict_as_defective.csv'
    
    if not os.path.exists(prediction_df_dir) or not os.path.exists(correctly_predict_df_dir):
        global_model = pickle.load(open(proj_name+'_'+global_model_name+'_global_model.pkl','rb'))

        pred = global_model.predict(x_test)
        defective_prob = global_model.predict_proba(x_test)[:,1]

        prediction_df = x_test.copy()
        prediction_df['pred'] = pred
        prediction_df['defective_prob'] = defective_prob
        prediction_df['defect'] = y_test

    #     print('AUC is',roc_auc_score(y_test, defective_prob))
        correctly_predict_df = prediction_df[(prediction_df['pred']==1) & (prediction_df['defect']==1)]

        print('total correct prediction: {}'.format(str(len(correctly_predict_df))))

        prediction_df.to_csv(prediction_df_dir)
        correctly_predict_df.to_csv(correctly_predict_df_dir)
    
    else:
        prediction_df = pd.read_csv(prediction_df_dir)
        correctly_predict_df = pd.read_csv(correctly_predict_df_dir)
        
        prediction_df = prediction_df.set_index('commit_id')
        correctly_predict_df = correctly_predict_df.set_index('commit_id')
        print('total correct prediction: {}'.format(str(len(correctly_predict_df))))
        
    return prediction_df, correctly_predict_df

def get_recall_at_k_percent_effort(percent_effort, result_df_arg, real_buggy_commits):
    cum_LOC_k_percent = (percent_effort/100)*result_df_arg.iloc[-1]['cum_LOC']
    buggy_line_k_percent =  result_df_arg[result_df_arg['cum_LOC'] <= cum_LOC_k_percent]
    buggy_commit = buggy_line_k_percent[buggy_line_k_percent['defect']==True]
    recall_k_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))
    
    return recall_k_percent_effort

def eval_global_model(proj_name, prediction_df):
    ## since ld metric in openstack is removed by using autospearman, so this code is needed
    ## but this is not problem for qt
    
    if proj_name == 'openstack':
        x_train_original, x_test_original = prepare_data_all_metrics(proj_name, mode='all')
        prediction_df = prediction_df.copy()
#         print('add ld')
#         display(x_test_original['ld'])
        prediction_df['ld'] = list(x_test_original['ld'])
        
    prediction_df = prediction_df[['la','ld', 'pred', 'defective_prob' ,'defect']]
    prediction_df['LOC'] = prediction_df['la']+prediction_df['ld']
    
    
#     result_df['defect_density'] = result_df['defective_commit_prob']/result_df['LOC']
    prediction_df['defect_density'] = prediction_df['defective_prob']/prediction_df['LOC']
    prediction_df['actual_defect_density'] = prediction_df['defect']/prediction_df['LOC'] #defect density
    
    prediction_df = prediction_df.fillna(0)
    prediction_df = prediction_df.replace(np.inf, 0)
    
    prediction_df = prediction_df.sort_values(by='defect_density',ascending=False)
#     display(prediction_df.head())
#     display(np.sum(prediction_df[prediction_df['la']==0]['defect']))
    
    actual_result_df = prediction_df.sort_values(by='actual_defect_density',ascending=False)
    actual_worst_result_df = prediction_df.sort_values(by='actual_defect_density',ascending=True)

    prediction_df['cum_LOC'] = prediction_df['LOC'].cumsum()
    actual_result_df['cum_LOC'] = actual_result_df['LOC'].cumsum()
    actual_worst_result_df['cum_LOC'] = actual_worst_result_df['LOC'].cumsum()

    real_buggy_commits = prediction_df[prediction_df['defect'] == True]
    
#     display(prediction_df)
#     display(real_buggy_commits)
    
    
    AUC = roc_auc_score(prediction_df['defect'], prediction_df['defective_prob'])
    f1 = f1_score(prediction_df['defect'], prediction_df['pred'])
    
    ifa = real_buggy_commits.iloc[0]['cum_LOC']
#     print('ifa:',ifa)

    cum_LOC_20_percent = 0.2*prediction_df.iloc[-1]['cum_LOC']
    buggy_line_20_percent = prediction_df[prediction_df['cum_LOC'] <= cum_LOC_20_percent]
    buggy_commit = buggy_line_20_percent[buggy_line_20_percent['defect']==True]
    recall_20_percent_effort = len(buggy_commit)/float(len(real_buggy_commits))
    
    # find P_opt
    percent_effort_list = []
    predicted_recall_at_percent_effort_list = []
    actual_recall_at_percent_effort_list = []
    actual_worst_recall_at_percent_effort_list = []
    
    for percent_effort in np.arange(10,101,10):
        predicted_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, prediction_df, real_buggy_commits)
        actual_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_result_df, real_buggy_commits)
        actual_worst_recall_k_percent_effort = get_recall_at_k_percent_effort(percent_effort, actual_worst_result_df, real_buggy_commits)
        
        percent_effort_list.append(percent_effort/100)
        
        predicted_recall_at_percent_effort_list.append(predicted_recall_k_percent_effort)
        actual_recall_at_percent_effort_list.append(actual_recall_k_percent_effort)
        actual_worst_recall_at_percent_effort_list.append(actual_worst_recall_k_percent_effort)

    p_opt = 1 - ((auc(percent_effort_list, actual_recall_at_percent_effort_list) - 
                 auc(percent_effort_list, predicted_recall_at_percent_effort_list)) /
                (auc(percent_effort_list, actual_recall_at_percent_effort_list) -
                auc(percent_effort_list, actual_worst_recall_at_percent_effort_list)))

    print('AUC: {}, F1: {}, IFA: {}, Recall@20%Effort: {}, Popt: {}'.format(AUC,f1,ifa,recall_20_percent_effort,p_opt))
    print(classification_report(prediction_df['defect'], prediction_df['pred']))
#     display(cum_LOC_20_percent)
#     display(buggy_line_20_percent)
# #     display(cum_LOC_20_percent)
#     display(buggy_commit)
#     print(len(real_buggy_commits))
#     display(recall_20_percent_effort)

def get_global_model_evaluation_result(proj_name):
    print('RF global model result')
    rf_prediction_df, rf_correctly_predict_df = get_prediction_result_df(proj_name, 'rf')
    eval_global_model(proj_name, rf_prediction_df)

    print('-'*100)
    
    print('LR global model result')
    lr_prediction_df, lr_correctly_predict_df = get_prediction_result_df(proj_name, 'lr')
    eval_global_model(proj_name, lr_prediction_df)


# # RQ3 evaluation

# In[31]:


def eval_rule(rule, x_df):
    var_in_rule = list(set(re.findall('[a-zA-Z]+', rule)))
    
    rule = re.sub(r'\b=\b','==',rule)
    if 'or' in var_in_rule:
        var_in_rule.remove('or')
        
    rule = rule.replace('&','and')
    
    eval_result_list = []
    
#     print(rule)

    for i in range(0,len(x_df)):
        x = x_df.iloc[[i]]
        col = x.columns
        var_dict = {}

        for var in var_in_rule:
            var_dict[var] = float(x[var])

#         print(var_dict)
        
        # if the rule does not satisfy clean commit, the truth value of the inversed rule when applied to clean commit is true
        eval_result = eval(rule,var_dict)
        eval_result_list.append(eval_result)
        
#         print(eval_result)
#         break
        
    return eval_result_list

# def summarize_rule_eval_result(py_exp_rule_str, lime_rule_str, x_df, ground_truth):
# #     print('Rulefit')
#     py_exp_all_eval_result = eval_rule(py_exp_rule_str, x_df)
# #     print('LIME')
#     lime_all_eval_result = eval_rule(lime_rule_str, x_df)

# #     print(py_exp_rule_str)
# #     print(lime_rule_str)
    
# #     tmp_df = x_df.copy()
# #     tmp_df['ground_truth'] = ground_truth
# #     tmp_df_clean = tmp_df[tmp_df['ground_truth']==False]
    
# #     display(tmp_df_clean)
    
#     py_exp_result_df = pd.DataFrame()
#     py_exp_result_df['ground_truth'] = ground_truth
#     py_exp_result_df['rule_result'] = py_exp_all_eval_result
#     py_exp_result_df = py_exp_result_df[py_exp_result_df['rule_result']==True] # get commit that matches counter rule
    
# #     print('py_exp_result_df')
# #     display(py_exp_result_df[py_exp_result_df['ground_truth']==False])
# #     print(len(py_exp_result_df))
#     # find ratio of clean commit
#     py_exp_satisfy_rule_ratio = 100*(len(py_exp_result_df[py_exp_result_df['ground_truth']==False])/len(py_exp_result_df)) if len(py_exp_result_df) > 0 else 0
    
#     lime_result_df = pd.DataFrame()
#     lime_result_df['ground_truth'] = ground_truth
#     lime_result_df['rule_result'] = lime_all_eval_result
    
#     lime_result_df = lime_result_df[lime_result_df['rule_result']==True] # get commit that matches counter rule
    
# #     print(len(lime_result_df))
    
#     # find ratio of clean commit
# #     print('lime_result_df')
# #     display(lime_result_df[lime_result_df['ground_truth']==False])
#     lime_satisfy_rule_ratio = 100*(len(lime_result_df[lime_result_df['ground_truth']==False])/len(lime_result_df))  if len(lime_result_df) > 0 else 0
    
# #     print(len(py_exp_result_df[py_exp_result_df['ground_truth']==False]))
# #     print(len(lime_result_df[lime_result_df['ground_truth']==False]))
    
#     return py_exp_satisfy_rule_ratio, lime_satisfy_rule_ratio

def summarize_rule_eval_result(rule_str, x_df):
#     print('Rulefit')
    all_eval_result = eval_rule(rule_str, x_df)
    all_eval_result = np.array(all_eval_result).astype(bool)
    
#     result_df = pd.DataFrame()
#     result_df['ground_truth'] = ground_truth
#     result_df['rule_result'] = all_eval_result
#     result_df = result_df[result_df['rule_result']==True] # get commit that matches counter rule
    
#     print('py_exp_result_df')
#     display(py_exp_result_df[py_exp_result_df['ground_truth']==False])
#     print(len(py_exp_result_df))
    # find ratio of clean commit
#     satisfy_rule_ratio = 100*(len(result_df[result_df['ground_truth']==False])/len(result_df)) if len(result_df) > 0 else 0

    return all_eval_result

'''
    input:
        local_model: local model of RuleFit
        X_explain: an instance to be explained
        
    return:
        g2_guide, g4_guide (string)
        more info of g2/g4 guidance refers to SQAPlanner paper
'''

def get_g2_g4_guidance(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] < 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g2_guide_df = rules[rules['is_satisfy_instance']==True]
    g4_guide_df = rules[rules['is_satisfy_instance']==False]

    g2_guide_df = g2_guide_df.sort_values(by='importance', ascending=False)
    g4_guide_df = g4_guide_df.sort_values(by='importance', ascending=False)
    
    g2_guide = g2_guide_df.iloc[0]['rule']
    g4_guide = g4_guide_df.iloc[0]['rule']
    
    return g2_guide, g4_guide


# In[32]:


def test_rule(proj_name):
    global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_namez)
    x_test, y_test = prepare_data(proj_name, mode = 'test')
    
    rq3_eval_result = pd.DataFrame() # for train data

    py_exp_guide = []
    lime_guide = []
    
    for i in range(0,len(feature_df)):
        X_explain = feature_df.iloc[[i]]

        row_index = str(X_explain.index[0])

        py_exp = pickle.load(open(pyExp_dir+proj_name+'_rulefit_crossoverinterpolation_'+row_index+'.pkl','rb'))
        lime_exp = pickle.load(open(pyExp_dir+proj_name+'_lime_'+row_index+'.pkl','rb'))

        py_exp_local_model = py_exp['local_model']
        lime_exp_local_model = lime_exp['local_model']
        
        py_exp_the_best_defective_rule_str = get_rule_str_of_rulefit(py_exp_local_model)
#         print(py_exp_the_best_defective_rule_str)
        
        total_cond_in_py_exp = len(py_exp_the_best_defective_rule_str.split('&'))
        
        lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

        py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
        lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

#         print(py_exp_pred, lime_pred)
        if py_exp_pred:
            py_exp_the_best_defective_rule_str = flip_rule(py_exp_the_best_defective_rule_str)
            py_exp_guide.append(py_exp_the_best_defective_rule_str)

        if lime_pred:
            lime_the_best_defective_rule_str = flip_rule(lime_the_best_defective_rule_str)
            lime_guide.append(lime_the_best_defective_rule_str)
        
    print(set(py_exp_guide))
    print('total guidance:',len(set(py_exp_guide)))
    print('-'*100)
    print(set(lime_guide))
    print('total guidance:',len(set(lime_guide)))


# In[33]:


def eval_PyExplainer_guidance(proj_name, global_model, method_name, row_index, guidance, x_test, y_test_flip , flip=False):
    guidance_list = guidance.split('&')
    
    guide_eval_result = pd.DataFrame()
    
    for condition in guidance_list:
        if flip:
            condition = flip_rule(condition)
            
        py_exp_guidance_eval = summarize_rule_eval_result(condition, x_test)

        guide_prec = precision_score(y_test_flip, py_exp_guidance_eval)
        guide_rec = recall_score(y_test_flip, py_exp_guidance_eval)

        py_exp_serie_test = pd.Series(data=[proj_name, row_index, method_name, global_model,condition, guide_prec, guide_rec])
        guide_eval_result = guide_eval_result.append(py_exp_serie_test,ignore_index=True)
        
    return guide_eval_result


# In[34]:
def get_g1_guidance(local_model, X_explain):
    rules = local_model.get_rules()
    rules = rules[(rules['type']=='rule') & (rules['coef'] > 0) & (rules['importance'] > 0)]
    rules_list = list(rules['rule'])
    
    rule_eval_result = []

    for r in rules_list:
        py_exp_pred = eval_rule(r, X_explain)[0]
        rule_eval_result.append(py_exp_pred)
        
    rules['is_satisfy_instance'] = rule_eval_result
    
    g1_guide_df = rules[rules['is_satisfy_instance']==True]

    g1_guide_df = g1_guide_df.sort_values(by='importance', ascending=False)
    
    g1_guide = g1_guide_df.iloc[0]['rule']
    
    return g1_guide

def rq3_eval(proj_name, global_model_name):
    global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)
    x_test, y_test = prepare_data(proj_name, mode = 'test')
    
    y_test_flip = [False if val else True for val in y_test]
    
    rq3_explanation_result = pd.DataFrame()
    
    pyexp_guidance_result_list = []
    lime_guidance_result_df = pd.DataFrame()
    
    for i in range(0,len(feature_df)):

        X_explain = feature_df.iloc[[i]]

        row_index = str(X_explain.index[0])

        exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
        py_exp = exp_obj['pyExplainer']
        lime_exp = exp_obj['LIME']

        # load local models
        py_exp_local_model = py_exp['local_model']
        lime_exp_local_model = lime_exp['local_model']
        
        # generate explanations                
        py_exp_the_best_defective_rule_str = get_g1_guidance(py_exp_local_model, X_explain)
        lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

        # check whether explanations apply to the instance to be explained
        py_exp_pred = eval_rule(py_exp_the_best_defective_rule_str, X_explain)[0]
        lime_pred = eval_rule(lime_the_best_defective_rule_str, X_explain)[0]

        # split rule to get list of conditions
        condition_list = py_exp_the_best_defective_rule_str.split('&')

        # for explanations
        for condition in condition_list:
            condition = condition.strip()

            py_exp_rule_eval = summarize_rule_eval_result(condition, x_test)

            rule_prec = precision_score(y_test, py_exp_rule_eval)
            rule_rec = recall_score(y_test, py_exp_rule_eval)

            py_exp_serie_test = pd.Series(data=[proj_name, row_index, 'pyExplainer',global_model_name, condition, rule_prec, rule_rec, py_exp_pred])
            rq3_explanation_result = rq3_explanation_result.append(py_exp_serie_test,ignore_index=True)

        # for guidance
        g2_guide, g4_guide = get_g2_g4_guidance(py_exp_local_model, X_explain)

        flip_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpFlip', row_index,
                                                           py_exp_the_best_defective_rule_str, x_test, y_test_flip, flip=True)
        g2_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG2', row_index,
                                                           g2_guide, x_test, y_test_flip, flip=False)
        g4_guide_eval_result = eval_PyExplainer_guidance(proj_name, global_model_name,'PyExpG4', row_index,
                                                           g4_guide, x_test, y_test_flip, flip=False)

        pyexp_guidance_result_list.append(flip_guide_eval_result)
        pyexp_guidance_result_list.append(g2_guide_eval_result)
        pyexp_guidance_result_list.append(g4_guide_eval_result)
        # PyExp END
        
       
        # LIME START
        lime_rule_eval = summarize_rule_eval_result(lime_the_best_defective_rule_str, x_test)

        rule_prec = precision_score(y_test, lime_rule_eval)
        rule_rec = recall_score(y_test, lime_rule_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME',global_model_name, lime_the_best_defective_rule_str, rule_prec, rule_rec, lime_pred])
        rq3_explanation_result = rq3_explanation_result.append(lime_serie_test,ignore_index=True)

        lime_guidance = flip_rule(lime_the_best_defective_rule_str)
        lime_guidance_eval = summarize_rule_eval_result(lime_guidance, x_test)
#             tn, fp, fn, tp = confusion_matrix(y_test, lime_rule_eval, labels=[1,0]).ravel()
#             tp_rate = tp/(tp+fn)
#             tn_rate = tn/(tn+fp)

        guide_prec = precision_score(y_test_flip, lime_guidance_eval)
        guide_rec = recall_score(y_test_flip, lime_guidance_eval)

        lime_serie_test = pd.Series(data=[proj_name, row_index, 'LIME', global_model_name, lime_guidance, guide_prec, guide_rec])
        lime_guidance_result_df = lime_guidance_result_df.append(lime_serie_test, ignore_index=True)
            
        print('finished {} from {} commits'.format(str(i+1),len(feature_df)))
        
        
    pyexp_guidance_result_df = pd.concat(pyexp_guidance_result_list)
    
    rq3_guidance_result = pd.concat([pyexp_guidance_result_df, lime_guidance_result_df])
    
    
    
    rq3_explanation_result.columns = ['project','commit_id','method','global_model','explanation','precision','recall', 'isSatisfy']
    rq3_guidance_result.columns = ['project','commit_id','method', 'global_model','guidance','precision','recall']

    rq3_explanation_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_explanation_eval_split_rulefit_condition.csv',index=False)
    rq3_guidance_result.to_csv(result_dir+'RQ3_'+proj_name+'_'+global_model_name+'_guidance_eval_split_rulefit_condition.csv',index=False)

    
    



In [ ]:
def get_combined_probs(X_input, global_model, input_guidance, input_SD):

    k_percent = 10
    prob_og = global_model.predict_proba(X_input)[0][1]


    output_df = pd.DataFrame()
    # 1) Revised values must not be negative -> if val < 0 then val = 0
    # 2) Revised values must not be lesser/greater than the actual values for < / > operations
    # E.g., {LOC < 50} => Clean and the actual LOC is 30: 
    # For a 10-percent decrease approach, the revised value according to the threshold is 45
    # In this case (Actual < Revised), we apply the 10-percent decrease approach to the actual value instead of the threshold.
    # Thus, the final revised value is 27 (10-percent decrease from 30)
    # There are 2 approaches:
    # (1) n-percent increase/decrease, e.g., 10-percent
    # (2) an SD_train increase/decrease, e.g., 1SD

    X_revised_percent = X_input.copy()
    X_revised_sd = X_input.copy()
    for guidance_i in input_guidance.split('&'):
        tmp_g = guidance_i.strip().split(' ')
#         print(tmp_g)
#         print(len(tmp_g))
        if len(tmp_g) != 3:
            continue

        revised_var = tmp_g[0]    
        revised_opr = tmp_g[1] 
        actual_val = X_input[revised_var][0]
        for approach_i in ['percent', 'sd']:

#             print('Approach', approach_i)
            if approach_i == 'percent':

                if '<' in revised_opr:
                    # < or <=
                    revised_from_threshold = float(tmp_g[2]) * (100 - k_percent) / 100.0
                    revised_from_actual = X_explain[revised_var][0] * (100 - k_percent) / 100.0
                    # the revised value from threshold must not greater than the actual values
                    if revised_from_threshold > actual_val:
                        actual_revised_value = revised_from_actual
                    else:
                        actual_revised_value = revised_from_threshold
                else:
                    # > or >=
                    revised_from_threshold = float(tmp_g[2]) * (100 + k_percent) / 100.0
                    revised_from_actual = X_explain[revised_var][0] * (100 + k_percent) / 100.0
                    # the revised value from threshold must not less than the actual values
                    if revised_from_threshold < actual_val:
                        actual_revised_value = revised_from_actual
                    else:
                        actual_revised_value = revised_from_threshold
                   
                # the revised value must not be negative
                if actual_revised_value < 0:
                    actual_revised_value = 0
                X_revised_percent[revised_var] = actual_revised_value
                
            else: #SD approach

                if '<' in revised_opr:
                    # < or <=
                    revised_from_threshold = float(tmp_g[2]) - input_SD[revised_var]
                    revised_from_actual = X_explain[revised_var][0] - input_SD[revised_var]
                    # the revised value from threshold must not greater than the actual values
                    if revised_from_threshold > actual_val:
                        actual_revised_value = revised_from_actual
                    else:
                        actual_revised_value = revised_from_threshold
                else:
                    # > or >=
                    revised_from_threshold = float(tmp_g[2]) + input_SD[revised_var]
                    revised_from_actual = X_explain[revised_var][0] + input_SD[revised_var]
                    # the revised value from threshold must not less than the actual values
                    if revised_from_threshold < actual_val:
                        actual_revised_value = revised_from_actual
                    else:
                        actual_revised_value = revised_from_threshold

                # the revised value must not be negative
                if actual_revised_value < 0:
                    actual_revised_value = 0
                
                X_revised_sd[revised_var] = actual_revised_value

    prob_revised_percent = global_model.predict_proba(X_revised_percent)[0][1]
    prob_revised_sd = global_model.predict_proba(X_revised_sd)[0][1]
    
    tmp_out = pd.Series(data=[input_guidance, prob_og, prob_revised_percent, prob_revised_sd])
    output_df = output_df.append(tmp_out,ignore_index=True)
    if len(output_df) == 0:
        return []
    output_df.columns = ['guidance', 'probOg', 'probRevisedPercent', 'probRevisedSD']
    return output_df

In [ ]:
def what_if_analysis(proj_name, global_model_name):
    global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)

    x_train, x_test, y_train, y_test = prepare_data(proj_name, mode = 'all')
    y_test_flip = [False if val else True for val in y_test]

    rq3_explanation_result = pd.DataFrame()

    pyexp_guidance_result_list = []
    lime_guidance_result_df = pd.DataFrame()

    x_train_sd = x_train.std()
    
    all_df = pd.DataFrame()

    get_revised_probs = get_combined_probs

    for i in range(0,len(feature_df)):

        tmp_df = pd.DataFrame()

        X_explain = feature_df.iloc[[i]]

        row_index = str(X_explain.index[0])

        exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
        py_exp = exp_obj['pyExplainer']
        lime_exp = exp_obj['LIME']

        # load local models
        py_exp_local_model = py_exp['local_model']
        lime_exp_local_model = lime_exp['local_model']

        # generate explanations                
        py_exp_the_best_defective_rule_str = get_g1_guidance(py_exp_local_model, X_explain)
        lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]

        # generate guidance
        ## PyExplainer - PyFlip, PyG2, PyG4

        pyFlip = flip_rule(py_exp_the_best_defective_rule_str)
        pyG2, pyG4 = get_g2_g4_guidance(py_exp_local_model, X_explain)


        tmp_df_i = get_revised_probs(X_explain, global_model, pyFlip, x_train_sd)
        if len(tmp_df_i) > 0:
            tmp_df_i['gtype'] = 'pyFlip'
            tmp_df = tmp_df.append(tmp_df_i)

        tmp_df_i = get_revised_probs(X_explain, global_model, pyG2, x_train_sd)
        if len(tmp_df_i) > 0:
            tmp_df_i['gtype'] = 'pyG2'
            tmp_df = tmp_df.append(tmp_df_i)

        tmp_df_i = get_revised_probs(X_explain, global_model, pyG4, x_train_sd)
        if len(tmp_df_i) > 0:
            tmp_df_i['gtype'] = 'pyG4'
            tmp_df = tmp_df.append(tmp_df_i)


        # LIME START
        lime_guidance = flip_rule(lime_the_best_defective_rule_str)

        tmp_df_i = get_revised_probs(X_explain, global_model, lime_guidance, x_train_sd)
        if len(tmp_df_i) > 0:
            tmp_df_i['gtype'] = 'LIMEFlip'
            tmp_df = tmp_df.append(tmp_df_i)

        tmp_df['commit_id'] = row_index
        tmp_df['model'] = global_model_name
        tmp_df['project'] = proj_name
        all_df = all_df.append(tmp_df)
        
    rq4_result_df.to_csv(result_dir+'RQ4_'+proj_name+'_'+global_model_name+'_guidance_eval_no_dup_original_rulefit_condition_new.csv',index=False)

    all_df.to_csv(result_dir+'RQ4_'+proj_name+'_'+global_model_name+'_combined_prob_from_guidance.csv',index=False)

In [3]:
proj_name = 'openstack'
global_model_name = 'RF'


In [23]:
global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)

x_train, y_train = prepare_data(proj_name, mode = 'train')

x_test, y_test = prepare_data(proj_name, mode = 'test')

y_test_flip = [False if val else True for val in y_test]

rq3_explanation_result = pd.DataFrame()

pyexp_guidance_result_list = []
lime_guidance_result_df = pd.DataFrame()

x_train_sd = x_train.std()

In [16]:
all_df = pd.DataFrame()

get_revised_probs = get_combined_probs

for proj_name in ['openstack', 'qt']:
    
    print(proj_name, 'START')
    for global_model_name in ['RF', 'LR']:
        print(global_model_name, 'START')
        
        global_model, correctly_predict_df, indep, dep, feature_df = prepare_data_for_testing(proj_name, global_model_name)

        x_train, y_train = prepare_data(proj_name, mode = 'train')

        x_test, y_test = prepare_data(proj_name, mode = 'test')

        y_test_flip = [False if val else True for val in y_test]

        rq3_explanation_result = pd.DataFrame()

        pyexp_guidance_result_list = []
        lime_guidance_result_df = pd.DataFrame()

        x_train_sd = x_train.std()
        
        for i in range(0,len(feature_df)):
            
            print('index', i)
            tmp_df = pd.DataFrame()

            X_explain = feature_df.iloc[[i]]

            row_index = str(X_explain.index[0])

            exp_obj = pickle.load(open(pyExp_dir+proj_name+'_'+global_model_name+'_all_explainer_'+row_index+'.pkl','rb'))
            py_exp = exp_obj['pyExplainer']
            lime_exp = exp_obj['LIME']

            # load local models
            py_exp_local_model = py_exp['local_model']
            lime_exp_local_model = lime_exp['local_model']

            # generate explanations                
            py_exp_the_best_defective_rule_str = get_g1_guidance(py_exp_local_model, X_explain)
            lime_the_best_defective_rule_str = lime_exp['rule'].as_list()[0][0]


            # generate guidance
            ## PyExplainer - PyFlip, PyG2, PyG4

            pyFlip = flip_rule(py_exp_the_best_defective_rule_str)
            pyG2, pyG4 = get_g2_g4_guidance(py_exp_local_model, X_explain)


            tmp_df_i = get_revised_probs(X_explain, global_model, pyFlip, x_train_sd)
            if len(tmp_df_i) > 0:
                tmp_df_i['gtype'] = 'pyFlip'
                tmp_df = tmp_df.append(tmp_df_i)

            tmp_df_i = get_revised_probs(X_explain, global_model, pyG2, x_train_sd)
            if len(tmp_df_i) > 0:
                tmp_df_i['gtype'] = 'pyG2'
                tmp_df = tmp_df.append(tmp_df_i)

            tmp_df_i = get_revised_probs(X_explain, global_model, pyG4, x_train_sd)
            if len(tmp_df_i) > 0:
                tmp_df_i['gtype'] = 'pyG4'
                tmp_df = tmp_df.append(tmp_df_i)


            # LIME START
            lime_guidance = flip_rule(lime_the_best_defective_rule_str)

            tmp_df_i = get_revised_probs(X_explain, global_model, lime_guidance, x_train_sd)
            if len(tmp_df_i) > 0:
                tmp_df_i['gtype'] = 'LIMEFlip'
                tmp_df = tmp_df.append(tmp_df_i)

            tmp_df['commit_id'] = row_index
            tmp_df['model'] = global_model_name
            tmp_df['project'] = proj_name
            all_df = all_df.append(tmp_df)

openstack START
RF START
index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
index 36
index 37
index 38
index 39
index 40
index 41
index 42
index 43
index 44
index 45
index 46
index 47
index 48
index 49
index 50
index 51
index 52
index 53
index 54
index 55
index 56
index 57
index 58
index 59
index 60
index 61
index 62
index 63
index 64
index 65
index 66
index 67
index 68
index 69
index 70
index 71
index 72
index 73
index 74
index 75
index 76
index 77
index 78
index 79
index 80
index 81
index 82
index 83
index 84
index 85
index 86
index 87
index 88
index 89
index 90
index 91
index 92
index 93
index 94
index 95
index 96
index 97
index 98
index 99
index 100
index 101
index 102
index 103
index 104
index 105
index 106
index 107
index

index 203
index 204
index 205
index 206
index 207
index 208
index 209
index 210
index 211
index 212
index 213
index 214
index 215
index 216
index 217
index 218
index 219
index 220
index 221
index 222
index 223
index 224
index 225
index 226
index 227
index 228
index 229
index 230
index 231
index 232
index 233
index 234
index 235
index 236
index 237
index 238
index 239
index 240
index 241
index 242
index 243
index 244
index 245
index 246
index 247
index 248
index 249
index 250
index 251
index 252
index 253
index 254


In [14]:
tmp_df

,guidance,probOg,probRevisedPercent,probRevisedSD,gtype,commit_id,model,project
0,age > 12.769999980926514,0.57,0.11,0.09,pyFlip,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,rtime > -7.359999895095825,0.57,0.66,0.57,pyG2,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,asawr > 0.004999999888241291,0.57,0.69,0.48,pyG4,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,la <= 58.00,0.57,0.22,0.09,LIMEFlip,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack


In [12]:
tmp_df_i

,guidance,probOg,probRevisedPercent,probRevisedSD,gtype
0,la <= 58.00,0.6,0.73,0.18,LIMEFlip


In [17]:
all_df

,guidance,probOg,probRevisedPercent,probRevisedSD,gtype,commit_id,model,project
0,la <= 3.4800000190734863 & age <= 0.5099999904...,0.570000,0.110000,0.090000,pyFlip,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,nrev > 1.0049999952316284 & nrev > 1.829999983...,0.570000,0.660000,0.570000,pyG2,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,nrev <= 2.100000023841858 & asawr > 0.00499999...,0.570000,0.690000,0.480000,pyG4,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,la <= 58.00,0.570000,0.220000,0.090000,LIMEFlip,1e6973aee7137653c62dcef970b1e2527b50517d,RF,openstack
0,la <= 102.78000259399414 & age > 3.23000001907...,0.580000,0.380000,0.270000,pyFlip,5d0ccceb20780fdd3adf519d3f8e6b80b1844407,RF,openstack
...,...,...,...,...,...,...,...,...
0,la <= 31.00,0.609164,0.602519,0.601450,LIMEFlip,979a0406f0013560efbdcc486b32ba93ce8c946f,LR,qt
0,age > 30.4399995803833 & nrev <= 3.21000003814...,0.675246,0.473491,0.314228,pyFlip,337524714cad51934879d817564c5d58e6dbd0c0,LR,qt
0,rsawr <= 0.11499999836087227 & ndev <= 6.83999...,0.675246,0.667617,0.633723,pyG2,337524714cad51934879d817564c5d58e6dbd0c0,LR,qt
0,hcmt <= 10.579999923706055 & age > 16.57500076...,0.675246,0.542494,0.345836,pyG4,337524714cad51934879d817564c5d58e6dbd0c0,LR,qt


In [18]:
all_df.to_csv(result_dir+'RQ4_combined_prob_from_guidance.csv',index=False)

In [26]:
all_df.to_csv(result_dir+'RQ4_revised_prob_from_guidance.csv',index=False)